In [ ]:
from matplotlib import pyplot,image

shanthiPic = '/src/shanthiPictures/101ND750/DSC_0181.JPG'
img = image.imread(shanthiPic)
imgplot=pyplot.imshow(img)
pyplot.show()

In [4]:
import hashlib
def calculateMD5Hash(file):
    fh = open(file, "rb")
    return hashlib.md5(fh.read()).hexdigest()
    

shanthiPic = '/src/data/testPythonDuplicateFinder/2015-09/IMG_1313.JPG'
print(calculateMD5Hash(shanthiPic))


FileNotFoundError: [Errno 2] No such file or directory: '/src/data/testPythonDuplicateFinder/2015-09/IMG_1313.JPG'

In [ ]:
### List Extention ###
from os import walk as oswalk
from os.path import join as osjoin
from time import time
inputPath = '/src/data/testPythonDuplicateFinder'
startTime = time()
imageHashes = []
images =[]
for root, dirs, files in oswalk(inputPath):
    images = [osjoin(root, f) for f in files]
    imageHashes.extend([(image, calculateMD5Hash(image)) for image in images])

print('Took {}s for {} images'.format(time() - startTime, len(imageHashes)))
print ('images {}'.format(len(images)))


In [ ]:
import concurrent.futures
from os import walk as oswalk
from os.path import join as osjoin
from time import time
import hashlib
def calculateMD5Hash(file):
    fh = open(file, "rb")
    return hashlib.md5(fh.read()).hexdigest()
    
inputPaths = ['/src/data/testPythonDuplicateFinder']
duplicatesPath = '/src/data/duplicates'
inputImages = []
for inputPath in inputPaths:
    for root, dirs, files in oswalk(inputPath):
        inputImages.extend(img for img in [osjoin(root, f) for f in files] if img not in inputImages)


startTime = time()
# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    # Start the load operations and mark each future with its URL
    future_to_file = {executor.submit(calculateMD5Hash, inputFile): inputFile for inputFile in inputImages}
    imageHashMap = {}
    for future in concurrent.futures.as_completed(future_to_file):
        file = future_to_file[future]
        try:
            md5Hash = future.result()
            
        except Exception as exc:
            print('%r generated an exception: %s' % (file, exc))
        else:
            if md5Hash in imageHashMap: 
                imageHashMap[md5Hash].extend([file])
            else:
                imageHashMap[md5Hash] = [file]

print (imageHashMap)
print('Took {}s for  {} images'.format(time() - startTime, len(inputImages)))


In [ ]:
import json
imageHashMapJson = json.JSONEncoder(sort_keys=True, indent=0).encode(imageHashMap)
print (imageHashMapJson)
with open('/src/data/fileHashes.json', 'w') as jsonFile:
    jsonFile.write(imageHashMapJson)

#json.dump(imageHashMap, '/src/data/fileHashes.json')

# two ways to log an exception

In [77]:
## Two ways to log exception
import traceback
try:
    print (os.path.normpath('/data'))
except :
    print (traceback.format_exc())
    
import logging
try:
    print (os.path.normpath('/data'))
except :
    logging.exception("hello")
    
    

ERROR:root:hello
Traceback (most recent call last):
  File "<ipython-input-77-8eb620650385>", line 9, in <module>
    print (os.path.normpath('/data'))
NameError: name 'os' is not defined


Traceback (most recent call last):
  File "<ipython-input-77-8eb620650385>", line 3, in <module>
    print (os.path.normpath('/data'))
NameError: name 'os' is not defined



# Setup the data

In [14]:
%%bash
## Data Setup
rm -rf /src/data/test1/ /src/data/test2/ /src/data/duplicates /src/data/*.json
cp -r '/src/data/test1 (copy)/' '/src/data/test1/'
cp -r '/src/data/test2 (copy)/' '/src/data/test2/'
cp -r '/src/data/test2 (copy)/' '/src/data/test2/2015-10/'
cp -r '/src/data/test1 (copy)/' '/src/data/test1/2015-09/'

# Code starts here

In [4]:
#%load_ext line_profiler
#%%time
## Start by loading the uniqueFiles list
## Prepare the list of unique file paths
## Find duplicates using the hash
## move duplicates to duplicates folder
## the run should output [hash, originalFilePath, [duplciate 1 path, /duplicate 2 path, duplicate 3,....]]
## save this uniqueFiles list along with the hash [hash, original file path]
import json
from os import walk, path, sep, makedirs

import concurrent.futures
import hashlib
import time
import shutil
import logging
import threading
makeDirLock = threading.Lock()

## loads the file paths and hashes that are previously known
## also ensures that all those files are actually present on the disk
def loadUniqueFilesMap(uniqueFilesMapFile):
    uniqueFilesMap = json.load(open(uniqueFilesMapFile)) if path.exists(uniqueFilesMapFile) else {}
     # We can use a with statement to ensure threads are cleaned up promptly
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Start the load operations and mark each future with its URL
        future_to_file = {executor.submit(path.exists, inputFile): inputFile for inputFile in uniqueFilesMap.values()}
        for future in concurrent.futures.as_completed(future_to_file):
            if (not future.result()):
                raise OSError("File not found : {}".format(future_to_file[future]))

    return uniqueFilesMap

## creates and returns a list of all the file paths from the inputPaths that are not
## known(not present in uniqueFilesMap)
def buildInputFilesList(inputPaths, uniqueFilesMap):
    inputFiles = []
    for inputPath in inputPaths:
        for root, dirs, files in walk(inputPath):
            inputFiles.extend(img for img in [path.join(root, f) for f in files] if img not in inputFiles if img not in uniqueFilesMap.values())
    return inputFiles

def calculateMD5Hash(file):
    fh = open(file, "rb")
    return hashlib.md5(fh.read()).hexdigest()

## find all the inputFiles that are already present in uniqueFilesMap and add them to the duplicateFilesMap
## all the new files frim teh inputFiles get added to uniqueFilesMap
def checkForDuplicates(inputFiles, uniqueFilesMap):
    duplicateFilesMap = {}
    # We can use a with statement to ensure threads are cleaned up promptly
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Start the load operations and mark each future with its URL
        future_to_file = {executor.submit(calculateMD5Hash, inputFile): inputFile for inputFile in inputFiles}
        for future in concurrent.futures.as_completed(future_to_file):
            file = future_to_file[future]
            md5Hash = future.result()
            if md5Hash in uniqueFilesMap:
                if md5Hash in duplicateFilesMap:
                    duplicateFilesMap[md5Hash].append(file)
                else:
                    duplicateFilesMap[md5Hash] = [uniqueFilesMap[md5Hash], file]
            else:
                uniqueFilesMap[md5Hash] = file
    return duplicateFilesMap


## Moves the inputFile to the duplicatesFolder
def moveFile(inputFile, duplicatesFolder):
    try:
        ## To join two absolute paths, we need to split the second path into its components and path
        ## that as an expanded list
        destFolder = path.join(duplicatesFolder, *path.normpath(path.dirname(inputFile)).split(sep))

        makeDirLock.acquire()
        ## Create the dest folder if necessary
        if (not path.exists(destFolder)):
            makedirs(destFolder)
        makeDirLock.release()

        shutil.move(inputFile, path.join(destFolder, path.basename(inputFile)))
        #print (inputFile)
    except:
        logging.exception("")
        raise
        
## moves all the duplicates from the duplicateFilesMap to duplicatesDestRoot folder and preserves
## the original directory structure
def moveTheDuplicates(duplicateFilesMap, duplicatesDestRoot):
    # We can use a with statement to ensure threads are cleaned up promptly
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Start the load operations and mark each future with its URL
        future_to_file = {}
        for duplicateFiles in duplicateFilesMap.values():
            if (len(duplicateFiles) > 1):
                ## This will merge the new dict returned by executer.submit with future_to_file and save the result
                ## in future_to_file
                future_to_file = {**future_to_file, **{executor.submit(moveFile, duplicateFile, duplicatesDestRoot): duplicateFile for duplicateFile in duplicateFiles[1:]}}
            else:
                raise Exception("how did you get a single file entry in duplicate files list?")
        for future in concurrent.futures.as_completed(future_to_file):
            pass
            
## Saves the duplicate and unique files list in a json file
def saveFileList(uniqueFilesMap, uniqueFilesMapPath, duplicateFilesMap, duplicateFilesMapPath):
    duplicateFilesMapJson = json.JSONEncoder(sort_keys=True, indent=0).encode(duplicateFilesMap)
    with open(duplicateFilesMapPath, 'w') as jsonFile:
        jsonFile.write(duplicateFilesMapJson)

   
    uniqueFilesMapJson = json.JSONEncoder(sort_keys=True, indent=0).encode(uniqueFilesMap)
    with open(uniqueFilesMapPath, 'w') as jsonFile:
        jsonFile.write(uniqueFilesMapJson)

def main():
    inputPaths=['/src/data/test1/', '/src/data/test2/']
    duplicatesPath='/src/data/duplicates'
    uniqueFilesMapPath='/src/data/uniqueFiles.json'
    duplicateFilesMapPath='/src/data/duplicateFiles.json'
    
    uniqueFilesMap=loadUniqueFilesMap(uniqueFilesMapPath)
    inputFiles = buildInputFilesList(inputPaths, uniqueFilesMap)
    duplicateFilesMap = checkForDuplicates(inputFiles, uniqueFilesMap)
    moveTheDuplicates(duplicateFilesMap, duplicatesPath)
    saveFileList(uniqueFilesMap, uniqueFilesMapPath, duplicateFilesMap, duplicateFilesMapPath)


#%lprun -f loadUniqueFilesMap main()

In [18]:
%lprun?

# Ensure duplicates are actually duplicates by checking the size 

In [11]:
import os
import json
import concurrent.futures

def haveSameSize(file1, file2):
    return os.stat(file1).st_size == os.stat(file2).st_size

## loads the file paths and hashes that are previously known
## also ensures that all those files are actually present on the disk
def loadDuplicateFilesMap(duplicateFilesMapFile):
    if os.path.exists(duplicateFilesMapFile):
        duplicateFilesMap = json.load(open(duplicateFilesMapFile))

        # We can use a with statement to ensure threads are cleaned up promptly
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_to_file = {}
            # Start the load operations and mark each future with its URL
            for duplicateFiles in duplicateFilesMap.values():
                if (len(duplicateFiles) < 1):
                    raise Exception("how did you get a single file entry in duplicate files list?")
                future_to_file = {**future_to_file, **{executor.submit(haveSameSize, duplicateFiles[0], os.path.join(duplicatesBasePath, *os.path.normpath(duplicateFile).split(os.sep))): duplicateFile for duplicateFile in duplicateFiles[1:]}}
            print ('{} future_to_file size'.format(len(future_to_file)))    
            for future in concurrent.futures.as_completed(future_to_file):
                if not future.result():
                    print ('{} is a false duplicate'.format(future_to_file[future]))

duplicatesBasePath = '/src/data/duplicates/'
loadDuplicateFilesMap('/src/data/duplicateFiles.json')

duplicatesBasePath = '/media/vijay/Data/Pictures/duplicates/'
loadDuplicateFilesMap('/media/vijay/Data/Pictures/logs/duplicateFiles.json')

print("done")

1115 future_to_file size
17605 future_to_file size
done


# UnitTest

# Create Image

In [45]:
import numpy as np
from PIL import Image, ImageColor
import os.path

def createTestData(basepath):
    imgArr = np.zeros(shape=[100,100,3], dtype=np.uint8)
    if not os.path.exists(basepath):
        os.makedirs(basepath)
    colorsAdded = set()
    for col in ImageColor.colormap.keys():
        colorString = ImageColor.colormap[col]
        if colorString not in colorsAdded:
            imgArr[:,:,:] = ImageColor.getrgb(colorString)
            Image.fromarray(imgArr, mode='RGB').save(os.path.join(basepath,'img{}.png'.format(col)))
            colorsAdded.add(colorString)
createTestData('/data/test')

In [9]:
import unittest

class TestStringMethods(unittest.TestCase):

    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # check that s.split fails when the separator is not a string
        with self.assertRaises(TypeError):
            s.split(2)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
#%tb

...
----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


In [51]:
import unittest
import os.path
import shutil

class SimpleTest(unittest.TestCase):
    basePath = '/data/test'
    def setUp(self):
        if os.path.exists(SimpleTest.basePath):
            shutil.rmtree(SimpleTest.basePath)
        createTestData(SimpleTest.basePath)
        
    def test_uniqueFiles(self):
        os.makedirs(os.path.join(SimpleTest.basePath,'1'))
        os.makedirs(os.path.join(SimpleTest.basePath,'2'))
        os.makedirs(os.path.join(SimpleTest.basePath,'3'))
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
#%tb

.
----------------------------------------------------------------------
Ran 1 test in 0.073s

OK
